In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
import sys, os

print(f"Installing packages into environment {sys.executable}")

In [ ]:
!{sys.executable} -m pip install llama-index openai langchain azure-identity

# Get Auth Token

In [ ]:
from azure.identity import DefaultAzureCredential

In [ ]:
# Request credential
default_credential = DefaultAzureCredential()

In [ ]:
token = default_credential.get_token("https://cognitiveservices.azure.com/.default")
# token.token

# Configure Parameters

In [ ]:
storage_account_url = "https://<storage account name>.blob.core.windows.net"

In [ ]:
aoai_base = "https://<aoai instance name>.openai.azure.com/"

In [ ]:
storage_container_name = "<blob storage container name>"

# Load documents from Azure Storage

In [ ]:
from llama_index import download_loader

In [ ]:
%%time
AzStorageBlobReader = download_loader("AzStorageBlobReader", refresh_cache = True)

In [ ]:
%%time
loader = AzStorageBlobReader(container_name=storage_container_name, account_url=storage_account_url, credential=default_credential)

In [ ]:
%%time
documents = loader.load_data()

In [ ]:
print('Number of documents: {}'.format(len(documents)))

# Setup Contexts

## Setup custom LLM

In [ ]:
from langchain.llms import AzureOpenAI

In [ ]:
azure_kwargs={"api_type": "azure_ad", "api_version": "2023-03-15-preview", "api_base": aoai_base}

In [ ]:
llm = AzureOpenAI(temperature=0.9, deployment_name="text-davinci-003", model_name="text-davinci-003", openai_api_key=token.token, model_kwargs=azure_kwargs)

In [ ]:
from llama_index import LLMPredictor

# define LLM
llm_predictor = LLMPredictor(llm)

In [ ]:
llm_predictor.get_llm_metadata()

### Embedding

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from llama_index import LangchainEmbedding

In [ ]:
os.environ["OPENAI_API_KEY"]=token.token

In [ ]:
# load in AOAI embedding model from langchain
oai_embeddings = OpenAIEmbeddings(model="text-embedding-ada-002",
                              deployment="text-embedding-ada-002",
                              openai_api_key=token.token,
                              openai_api_base=azure_kwargs["api_base"],
                              openai_api_type=azure_kwargs["api_type"],
                              openai_api_version=azure_kwargs["api_version"],
                              chunk_size=1)

In [ ]:
embeddings = LangchainEmbedding(oai_embeddings)

In [ ]:
from llama_index import ServiceContext, StorageContext

In [ ]:
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embeddings) # using default chunk limit

In [ ]:
storage_context = StorageContext.from_defaults() # No persist_dir means all stores will be new Simple stores

## Write document to store

In [ ]:
nodes = service_context.node_parser.get_nodes_from_documents(documents)

In [ ]:
storage_context.docstore.add_documents(nodes)

In [ ]:
storage_context.persist(persist_dir="./doc_store")

# Create Index

In [ ]:
from llama_index import GPTVectorStoreIndex

In [ ]:
%%time
vector_index = GPTVectorStoreIndex(nodes, service_context=service_context, storage_context=storage_context)

In [ ]:
storage_context.persist(persist_dir="./doc_store")

In [ ]:
# query will use the same embed_model
query_engine = vector_index.as_query_engine(
    verbose=True, 
)
response = query_engine.query("Please can you suggest the 5 best questions the reports address?")
print(response)